
# Data cleaning and feature engineering

---

In [8]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
# from sklearn.ensemble import RandomForestClassifier

<br>

## Load data and rename columns

#### Training data:

- 29k rows x 10 cols, 
- 'Beta' is outcome: methylated or not 
- Chromosomes 1-10

#### Test: 
- 20,611 rows, no outcome labels
- Chromosomes 11-22

In [9]:
train = pd.read_csv('data/train.csv')

# give the data names that don't suck
train = train.rename(columns={"Id": "id",
                              "CHR": "chromosome", 
                              "MAPINFO": "position",
                              "UCSC_CpG_Islands_Name": "island",  
                              "UCSC_RefGene_Group":"refgene",
                              "Relation_to_UCSC_CpG_Island": "rel_to_island",
                              "Regulatory_Feature_Group": "feature",
                              "Forward_Sequence":"fwd_seq",
                              "Beta": "outcome"})

# change categorical variables dtypes
for col in ["rel_to_island", "outcome"]:
    train[col] = train[col].astype("category")
for col in ["fwd_seq", "seq", "refgene"]:
    train[col] = train[col].astype("string")
train['position'] = train['position'].astype('float64')
del(col)

Y = train['outcome']
df = train.drop(['id', 'chromosome', 'outcome'], 1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29065 entries, 0 to 29064
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   position       29065 non-null  float64 
 1   island         23717 non-null  object  
 2   refgene        24822 non-null  string  
 3   rel_to_island  23717 non-null  category
 4   feature        16421 non-null  object  
 5   fwd_seq        29065 non-null  string  
 6   seq            29065 non-null  string  
dtypes: category(1), float64(1), object(2), string(3)
memory usage: 1.4+ MB


In [10]:
# df = df.iloc[:100, ]
# df.info()

---

<br>

## Features of the data 

- `id`: unique identifiers 
  - not useful for training, useful for data handling though  
  
  
- `chromosome, position` (exact CpG site): 
  - not really sure how to use this given that test is from different chromosomes  
  - don't want classifier to memorize positions (values = n)  
  - could derive other features like size of island, placement of cg inside of island  
    
    
- `island`: chr + position range of island  
  - again, very specific to each site which could lead to over training  
  - some missing data (4k)  
  
- `refgene`: UCSC_RefGene_Group
  - 1089 unique values...
  - contains list of tags about functional elements:
      - TSS* {200, .}
      - 1st exon
      - Body
      - 5'UTR
  - Each can have multiple tags, even multiple of same tags....
  - Maybe split into counts for each tag: columns [TSS200, TSS1500, exon1, body, utr5, ...]

Body:14797  TSS200:10935   5'UTR:9117 1stExon:6928 TSS1500:7634   3'UTR:866    NA's:4243 
                           

- `feature`: lots of missing data, seems like 2 different factors
  - Promoter/Gene/NonGene_Associated or Unclassified  
      - not Cell_type_specific
  - Promoter/Gene/NonGene_Associated_Cell_type_specific or Unclassified_Cell_type_specific
  - NA (~12k)
  
  
- `relation_to_island`:
  - 5 levels: Island:18269, S_Shore:2107, N_Shelf: 529, N_Shore: 2378, S_Shelf: 434, NA's: 5348
  
  
- `Fwd_seq` and `seq`:
  - not sure what is the relationship between these?
  - `Fwd_seq` has the [CG] site marked and are all 124 bp long
  - `seq` is 2kbp of sequence

  
----

## Nominal data
  
  <br>

In [11]:
# refgene has lists of tags
print(df['refgene'].unique()[:10])

<StringArray>
[                                'Body;Body',
                                    'TSS200',
                               "5'UTR;5'UTR",
                                      'Body',
                           '1stExon;1stExon',
                        "5'UTR;TSS200;5'UTR",
                                        <NA>,
                       'Body;Body;Body;Body',
 "5'UTR;1stExon;1stExon;5'UTR;1stExon;5'UTR",
                           'TSS1500;TSS1500']
Length: 10, dtype: string


In [12]:
# feature has 2 categories: classes (celltype_specific or not)
# and (promoter / gene / non-gene / unclassified)
print(df['feature'].unique())

[nan 'Promoter_Associated' 'Unclassified' 'Gene_Associated'
 'Unclassified_Cell_type_specific'
 'Promoter_Associated_Cell_type_specific' 'NonGene_Associated'
 'Gene_Associated_Cell_type_specific'
 'NonGene_Associated_Cell_type_specific']


In [13]:
# relation to island is one of 5 levels or unknown
print(list(df['rel_to_island'].unique()), '\n')


['Island', nan, 'S_Shore', 'N_Shelf', 'N_Shore', 'S_Shelf'] 



---

## Making dummy variables for categories

In [14]:
def make_dummies(df):
    """Make various dummies for (relation to island), (refgene group), (regulatory features)"""
    
    dfx = df.copy()
    ## get dummies for "Relation_to_UCSC_CpG_Island": 5 levels
    dfx = pd.get_dummies(dfx, columns =['rel_to_island'], prefix_sep = '', prefix = '')
    
    ## pull terms from 'UCSC_RefGene_Group' lists into columns of counts
    for term in ["TSS200", "TSS1500", "Body", "5'UTR", "3'UTR", "1stExon"]:
        dfx[term] = dfx["refgene"].str.count(term)
        dfx[term] = dfx[term].fillna(0).astype('int32')

    ## create 2 sets of dummies from 'feature' (Regulatory_Feature_Group)
    df["cell_type_specific"] = df['feature'].str.count("_Cell_type_specific").fillna(0).astype('int32')
    for term in ["Gene_Associated", "NonGene_Associated", "Promoter_Associated", "Unclassified"]:
        dfx[term] = dfx['feature'].str.count(term).fillna(0).astype('int32')

    dfx = dfx.drop(columns = ['position', 'island', 'refgene', 'feature', 'fwd_seq', 'seq'])
    return(dfx)

In [15]:
# dummies = make_dummies(df)
# dummies.describe()

---

## Position relative to nearby CpG Island

<br>


In [16]:
def make_relative_positions(df):
    """ island col has position info like: "chr1:2004858-2005346"
        We want to get the position of the CpG site relative to the start and stop of the island
        Many columns don't have island data so add a dummy to indicate whether it exists
    """
    dfx = df.copy()
    # dummy variable for whether has 'island' or NA
    dfx['has_island'] = np.where(dfx['island'].isna(), 0, 1)
    
    # postion of CpG relative to nearby island start position (lots of missing values though)
    dfx['isl_start'] = dfx['island'].str.extract(':(\d+)').astype('float64')
    dfx['dist_start'] = dfx['isl_start'] - dfx['position']
    dfx['dist_start'] = dfx['dist_start'].fillna(0)
    
    # same for distance to end of island
    dfx['isl_end'] = dfx['island'].str.extract('-(\d+)').astype('float64')
    dfx['dist_end'] = dfx['isl_end'] - df['position']
    dfx['dist_end'] = dfx['dist_end'].fillna(0)
    
    # return distance columns
    return(dfx[['has_island', 'dist_start', 'dist_end']])

In [17]:
# pos = make_relative_positions(df)
# pos.describe()

---

## Encoding sequences

There are shorter regions (60 bp up and downstream) and longer sequences (2kbp)

The shorter sequence always has 60 bp, then "[CG]", then 60 bp downstream.
```
print(df['short_l'][1])
print(set(len(x) for x in df['short_l']), set(len(x) for x in df['short_r']))
> GGACCACACTGCCATGGCAACAGCGTGCCTCTGCGTCCTCCATCCGGGCCTCTCTAACTA
> {60} {60}

```

The longer sequence contains the shorter in the center, always starting in the same position (939; from 0-index).
```
for x in range(10):
    pos = df['seq'][x].find(re.sub(r'[\[|\]]', '', df['fwd_seq'][x]))
    print(pos)
```

In [18]:
## with help from: https://www.kaggle.com/thomasnelson/working-with-dna-sequence-data-for-ml

def make_kmer_freq(df):
    """returns vectorized kmer frequency features as dataframe"""
    
    def get_kmers(dna, k=6):
        """creates list of kmers from dna seq"""
        dna = dna.upper()
        kmers = [dna[x:x+k] for x in range(len(dna)+1-k)]
        kmers = ' '.join(kmers)
        return(kmers)
    
    # create new column of 
    mers = df.apply(lambda x: get_kmers(x['seq'], 6), axis = 1)
    tfidf = TfidfVectorizer() 
    X = tfidf.fit_transform(mers)
    kmers = tfidf.get_feature_names()
    kmer_df = pd.DataFrame(X.toarray(), columns=kmers)
    return(kmer_df)


def make_one_hot_seq(df):
    
    def one_hot_encode_dna(dna):
        """ One-hot encode a single DNA sequence: 
        Requires creating two encoders: LabelEncoder to get from string to numeric, then OneHotEncoder
        Converts DNA to numeric then to one-hot matrix with shape: len(dna)*4
        """
        # create label encoder for DNA symbols
        label_encoder = LabelEncoder() 
        label_encoder.fit(np.array(list('ACGTN')))

        # create one-hot encoder
        onehot_encoder = OneHotEncoder(sparse=False, dtype=int)

        # dna to numeric array
        dna = re.sub('[^ACGT]', 'N', dna.upper())
        dna = np.array(list(dna))
        dna_int = label_encoder.transform(dna) 
        dna_int = dna_int.reshape(len(dna_int), 1)

        # convert to one-hot
        dna_onehot = onehot_encoder.fit_transform(dna_int)
        return(dna_onehot)
    
    """
    Splits the region around CpG site in up + downstream
    Applies one-hot encoding to each sequence and returns 480 column df
    """
    dfx = df.copy()
    # split the upstream and downstream seq around '[CpG]'; rejoin the two halves
    dfx['fwd_seq_x'] = dfx['fwd_seq'].str.split('\[|\]', expand = True).apply(lambda x: x[0] + x[2], axis=1)

    # apply one_hot_encoding to sequence and flatten matrix into vector
    X1 = dfx.apply(lambda x: one_hot_encode_dna(x['fwd_seq_x']).flatten(), axis=1)
    
    # stack vectors into data frame with 480 columns (x1 to x480), since [120 bp * 4 bases] are encoded.
    X1 = pd.DataFrame(np.column_stack(list(zip(*X1))), 
                      columns = list(x+str(y) for y in range(120) for x in 'ACGT'))

    return(X1)


In [19]:
# kmer_freq = make_kmer_freq(df)
# kmer_freq.info()

In [20]:
# one_hot_df = make_one_hot_seq(df)
# one_hot_df.info()

---

## Combine all the features together for modelling

In [22]:
def make_all_features(df):
    return(pd.concat([make_relative_positions(df.copy()),
                      make_dummies(df.copy()), 
                      make_kmer_freq(df.copy()), 
                      make_one_hot_seq(df.copy())], 
                     1)
          )


In [23]:
X = make_all_features(df)

In [ ]:
X.to_csv("data/train_X.csv")

In [ ]:
Y.to_csv("data/train_Y.csv")

In [ ]:
X.info()

In [ ]:
del(train)

----


# Test data

<br>

In [ ]:
## Same thing for test data
test = pd.read_csv('data/test.csv')
test = test.rename(columns={"Id": "id",
                              "CHR": "chromosome", 
                              "MAPINFO": "position",
                              "UCSC_CpG_Islands_Name": "island",  
                              "UCSC_RefGene_Group":"refgene",
                              "Relation_to_UCSC_CpG_Island": "rel_to_island",
                              "Regulatory_Feature_Group": "feature",
                              "Forward_Sequence":"fwd_seq"})

# change categorical variables dtypes
test["rel_to_island"] = test["rel_to_island"].astype("category")

In [ ]:
test